# Base CatBoost Regressor

In [1]:
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import tensorflow as tf

from collections import Counter
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn import ensemble, linear_model
from xgboost.sklearn import XGBClassifier
from bayes_opt import BayesianOptimization
from tensorflow import keras
from tensorflow.keras import layers
from catboost import Pool, CatBoostClassifier
from utils.metrics import Metric
from tqdm import tqdm
weights = pd.read_csv('data/005_weights.csv')['weight'].values

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

X_submit = pd.concat([ 
    pd.read_csv('./data/440_X_submit.csv'), 
    pd.read_csv('./data/430_X_submit.csv').drop('id', axis=1), 
    pd.read_csv('./data/510_X_submit_preds.csv', header=None),
    pd.read_csv('./data/520_X_submit_preds.csv', header=None),
    pd.read_csv('./data/522_X_submit_preds.csv', header=None)
], axis=1)

X_train = pd.concat([
    pd.read_csv('./data/440_X_train.csv'), 
    pd.read_csv('./data/430_X_train.csv').drop('id', axis=1),
    pd.read_csv('./data/510_X_train_preds.csv', header=None),
    pd.read_csv('./data/520_X_train_preds.csv', header=None),
    pd.read_csv('./data/522_X_train_preds.csv', header=None)
], axis=1)

X_test = pd.concat([
    pd.read_csv('./data/440_X_test.csv'),
    pd.read_csv('./data/430_X_test.csv').drop('id', axis=1),
    pd.read_csv('./data/510_X_test_preds.csv', header=None), 
    pd.read_csv('./data/520_X_test_preds.csv', header=None),
    pd.read_csv('./data/522_X_test_preds.csv', header=None)
], axis=1)

Num GPUs Available:  1


In [2]:
X_submit = X_submit.set_index('id')
X_train  = X_train.set_index('id')
X_test   = X_test.set_index('id')

y_submit = pd.read_csv('./data/004_test.csv', index_col='id')
y_train = pd.read_csv('./data/410_y_train.csv', index_col='id')
y_test = pd.read_csv('./data/410_y_test.csv', index_col='id')


X_submit = X_submit.values
X_train = X_train.values
X_test = X_test.values
y_train = np.squeeze(y_train.values)
y_test = np.squeeze(y_test.values)

X_train.shape

(80000, 447)

In [3]:
X_submit[:,:6] = X_submit[:,:6].astype(str)
X_train[:,:6] = X_train[:,:6].astype(str)
X_test[:,:6] = X_test[:,:6].astype(str)

In [12]:
y_train

array([ 9,  2,  3, ..., 10,  2, 12])

In [12]:
train_data = Pool(X_train, y_train, cat_features=[0,1,2,3,4,5])
test_data = Pool(X_test, y_test, cat_features=[0,1,2,3,4,5])

model = CatBoostClassifier(iterations=5000,
                           depth=4,
                           learning_rate=0.03,
                           loss_function='MultiClass',
                           random_seed=100,
                           one_hot_max_size=50,
                           verbose=True)

In [13]:
# train the model
model.fit(train_data, 
          eval_set=test_data, 
          early_stopping_rounds=35)

0:	learn: 2.4575303	test: 2.4546148	best: 2.4546148 (0)	total: 290ms	remaining: 24m 7s
1:	learn: 2.3703570	test: 2.3653604	best: 2.3653604 (1)	total: 587ms	remaining: 24m 25s
2:	learn: 2.2976284	test: 2.2911642	best: 2.2911642 (2)	total: 889ms	remaining: 24m 40s
3:	learn: 2.2328038	test: 2.2249615	best: 2.2249615 (3)	total: 1.17s	remaining: 24m 21s
4:	learn: 2.1762755	test: 2.1671186	best: 2.1671186 (4)	total: 1.45s	remaining: 24m 6s
5:	learn: 2.1262146	test: 2.1162967	best: 2.1162967 (5)	total: 1.76s	remaining: 24m 24s
6:	learn: 2.0817958	test: 2.0710799	best: 2.0710799 (6)	total: 2.06s	remaining: 24m 28s
7:	learn: 2.0408513	test: 2.0292443	best: 2.0292443 (7)	total: 2.36s	remaining: 24m 34s
8:	learn: 2.0036741	test: 1.9912426	best: 1.9912426 (8)	total: 2.65s	remaining: 24m 32s
9:	learn: 1.9694291	test: 1.9561819	best: 1.9561819 (9)	total: 2.94s	remaining: 24m 29s
10:	learn: 1.9369221	test: 1.9229892	best: 1.9229892 (10)	total: 3.22s	remaining: 24m 18s
11:	learn: 1.9071741	test: 1.892

In [14]:
df_submit = pd.DataFrame.from_records(model.predict_proba(X_submit))

cols = ['class'+str(i) for i in range(13)]

df_submit.columns = cols
df_submit.head()

,class0,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,class11,class12
0,0.004388,0.005987,0.268693,0.027795,0.086402,0.000045,0.003958,0.007880,0.004884,0.002027,0.531170,0.001448,0.055321
1,0.000857,0.007049,0.746808,0.115228,0.069057,0.000013,0.002329,0.009772,0.003443,0.009114,0.034279,0.000483,0.001569
2,0.001516,0.030800,0.112514,0.763034,0.007659,0.000012,0.008596,0.007813,0.015281,0.009396,0.031261,0.000154,0.011964
3,0.000771,0.100787,0.215823,0.322645,0.218926,0.000010,0.007986,0.039855,0.009397,0.012762,0.056495,0.002869,0.011673
4,0.006651,0.001857,0.026181,0.004660,0.020327,0.000003,0.000953,0.000298,0.000742,0.000491,0.932489,0.000329,0.005019


In [15]:
df_submit = pd.concat([
    y_submit.reset_index(),
    df_submit
], axis=1)

df_submit = df_submit.set_index('id')
df_submit

,class0,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,class11,class12
id,,,,,,,,,,,,,
151807,0.004388,0.005987,0.268693,0.027795,0.086402,4.536730e-05,0.003958,0.007880,0.004884,0.002027,0.531170,0.001448,0.055321
118131,0.000857,0.007049,0.746808,0.115228,0.069057,1.323127e-05,0.002329,0.009772,0.003443,0.009114,0.034279,0.000483,0.001569
110921,0.001516,0.030800,0.112514,0.763034,0.007659,1.196816e-05,0.008596,0.007813,0.015281,0.009396,0.031261,0.000154,0.011964
105149,0.000771,0.100787,0.215823,0.322645,0.218926,9.892000e-06,0.007986,0.039855,0.009397,0.012762,0.056495,0.002869,0.011673
143868,0.006651,0.001857,0.026181,0.004660,0.020327,3.077644e-06,0.000953,0.000298,0.000742,0.000491,0.932489,0.000329,0.005019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146316,0.001383,0.016119,0.069230,0.666364,0.172940,1.445683e-06,0.002787,0.009917,0.006679,0.011875,0.018662,0.000428,0.023614
121816,0.002497,0.001699,0.141787,0.020752,0.288340,1.423993e-05,0.000396,0.000142,0.000559,0.000315,0.539467,0.000555,0.003477
106217,0.000187,0.020435,0.017967,0.073731,0.848943,1.886945e-06,0.004102,0.001753,0.009350,0.001956,0.018282,0.001689,0.001603


In [16]:
df_submit.to_csv('013_submit.csv')

In [18]:
from joblib import dump, load

In [19]:
dump(model, 'M_336/623_stack_cat.joblib')

['M_336/623_stack_cat.joblib']